# Basic trajectory example

In [ ]:
import numpy as np
import pylab as plt
from pathlib import Path
import xarray as xr
import cartopy.crs as ccrs
import geopandas as gp
import pandas as pd
from tqdm.auto import tqdm

from pypism.trajectory import compute_perturbation
from pypism.hillshade import hillshade
from pypism.utils import qgis2cmap, tqdm_joblib, blend_multiply

np.seterr(divide='ignore', invalid='ignore')

In [ ]:
data_url = Path("~/Google Drive/My Drive/data/ITS_LIVE/GRE_G0240_0000.nc")
ogr_url = Path("/Users/andy/Google Drive/My Drive/data/GreenlandFluxGatesAschwanden/greenland-flux-gates-jib.shp")

In [ ]:
result = gp.read_file(Path("traj.gpkg"))

In [ ]:
run ../compute_pathlines.py --raster_url data_url --vector_url ogr_url

In [ ]:
qgis_colormap = Path("../data/speed-colorblind.txt")
cmap = qgis2cmap(qgis_colormap, name="speeds")

In [ ]:
ds_dem = xr.open_dataset(Path("/Users/andy/Google Drive/My Drive/data/MCdataset/BedMachineGreenland-v5.nc")).sel(x=slice(-210000, 200000), y=slice(-2100000, -2400000))

In [ ]:
ds_dem = xr.open_dataset(Path("/Users/andy/Google Drive/My Drive/data/MCdataset/BedMachineGreenland-v5.nc")).sel(x=slice(-210000, 200000), y=slice(-2100000, -2400000))
hs = hillshade(ds_dem["surface"], zf=5)
hs.plot(cmap="Greys_r", vmin=0, vmax=1, add_colorbar=False)

In [ ]:
n_jobs = 10
n_perturbations = 50
with tqdm_joblib(tqdm(desc="Processing Perturbation", total=n_perturbations)) as progress_bar:
    result = Parallel(n_jobs=n_jobs)(
        delayed(compute_perturbation)(data_url, ogr_url, perturbation=k, dt=10, total_time=10_000, reverse=True, pl_exp=2)
            for k in range(n_perturbations)
        )
results = pd.concat(result).reset_index(drop=True)

In [ ]:
crs = ccrs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70, globe=None)

jak_extent = {"x": slice(-240000, 60000), "y": slice(-2000000, -2350000)}

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection=crs)


f = ds.sel(jak_extent)["v"].plot(vmin=10, vmax=1500, cmap=cmap, shading="flat", alpha=0.5, ax=ax,
                 transform=crs, 
             cbar_kwargs={"location": "right", "orientation": "vertical", 
                          "fraction": 0.085, "shrink": 0.6, "label": ds["v"].units},
)
hs.plot(cmap="Greys_r", vmin=0, vmax=1, ax=f.axes, zorder=-1, add_colorbar=False)
f.axes.coastlines(linewidth=0.25, resolution="10m")

results.plot(ax=f.axes, markersize=0.1, column="perturbation", cmap="tab20b",
                  legend=False)
#stp.plot(ax=ax, color="k", lw=2)
f.axes.set_extent([-51, -40, 68., 70.5])

In [ ]:
results = pd.concat(result).reset_index(drop=True)

In [ ]:
ds = xr.open_dataset(data_url, decode_times=False)

In [ ]:
Path("foo/bar.txt").parent

In [ ]:
ds.vx_err.wher

In [ ]:
# Generates power-law power spectrum - structures of all sizes and fractal sub-structures
def plPk(n):
    def Pk(k):
        return np.power(k, -n)

    return Pk


def gaussPk(grf_range):
    def Pk(k):
        a = 2 / grf_range
        exponent = (k**2) / (4 * a)
        return np.sqrt(np.pi / a) * (np.e ** (-exponent))

    return Pk


# Draw samples from a normal distribution
def distrib(shape):
    a = np.random.normal(
        loc=0, scale=(2 * np.nanstd(jak.v)) ** 2, size=shape
    )
    b = np.random.normal(
        loc=0, scale=(2 * np.nanstd(jak.v)) ** 2, size=shape
    )
    return a + 1j * b


# Draw samples from a normal distribution
def distrib(shape):
    a = np.random.normal(
        loc=0, scale=jak.v.std(), size=shape
    )
    b = np.random.normal(
        loc=0, scale=jak.v.std(), size=shape
    )
    return a + 1j * b

def distrib_uniform(shape):
    a = np.random.uniform(size=shape)
    b = np.random.uniform(size=shape)
    vmin = jak.v - jak.v_err
    vmax = jak.v + jak.v_err
    return jak.v.to_numpy() + vmin.to_numpy() + (a + 1j * b) * (vmax-vmin).to_numpy()

def distrib_normal(shape):
    a = np.random.normal(
        loc=0, scale=jak.v_err, size=shape
    )
    b = np.random.normal(
        loc=0, scale=jak.v_err, size=shape
    )
    return a + 1j * b

## Just a helper to visualize the GRFs and thwat they do
def plot_gauss_rand_field(
    original_ds: xr.DataArray,
    grf: np.ndarray,
    err_field: xr.DataArray,
):
    fig, axs = plt.subplots(1, 3, figsize=(30, 10))
    ## Original data
    original_ds.plot.imshow(cmap="cividis", ax=axs[0], vmin=10, vmax=1000)
    ## Random Field
    grf_im = axs[1].imshow(grf, cmap="seismic")
    fig.colorbar(grf_im, ax=axs[1])
    ## Original data + Random Field
    err_field.plot.imshow(cmap="cividis", ax=axs[2], vmin=10, vmax=1000)

In [ ]:
ds = xr.open_dataset(data_url, decode_times=False)
jak_extent = {"x": slice(-240000, 60000), "y": slice(-2000000, -2350000)}
jak = ds.sel(jak_extent)

In [ ]:
shape = (len(jak.y.values), len(jak.x.values))
for pl_exp in np.linspace(0, 5, 6):
    grfield = generate_field(distrib_normal, plPk(pl_exp), shape)
    v_hat_field = jak.v + grfield
    plot_gauss_rand_field(jak.v, grfield, v_hat_field)
    plt.title(f"GRF with Scale Invariant Spectrum with exponent={pl_exp}")

In [ ]:
jak.v_err.plot(vmin=0, vmax=10)

In [ ]:
    a = np.random.normal(
        loc=0, scale=(2 * np.nanstd(jak.v)) ** 2, size=shape
    )

In [ ]:
from typing import Callable, Optional, Dict, Any

def generate_field_xr(
    fftfield: np.ndarray,
    power_spectrum: Callable[[np.ndarray], np.ndarray],
    unit_length: float = 1,
    fft: Any = np.fft,
    fft_args: Dict[str, Any] = {},
) -> np.ndarray:
    """
    Generates a field given a statistic and a power_spectrum.
    """

    if not isinstance(power_spectrum, Callable):
        raise Exception("`power_spectrum` should be callable")

    try:
        fftfreq = fft.fftfreq
    except AttributeError:
        # Fallback on numpy for the frequencies
        fftfreq = np.fft.fftfreq
    else:
        fftfreq = fft.fftfreq

    # Compute the k grid
    all_k = [fftfreq(s, d=unit_length) for s in shape]

    kgrid = np.meshgrid(*all_k, indexing="ij")
    knorm = np.hypot(*kgrid)

    power_k = np.zeros_like(knorm)
    mask = knorm > 0
    power_k[mask] = np.sqrt(power_spectrum(knorm[mask]))
    fftfield *= power_k

    return np.real(fft.ifftn(fftfield, **fft_args))

In [ ]:
np.random.uniform(size=shape)

In [ ]:
np.nanstd(jak.v)

In [ ]:
def distrib(shape):
    a = np.random.normal(
        loc=0, scale=jak.v.std(), size=shape
    )
    b = np.random.normal(
        loc=0, scale=jak.v.std(), size=shape
    )
    return a + 1j * b

In [ ]:
distrib_uniform(shape).max()

In [ ]:
def distrib_normal(shape):
    a = np.random.normal(
        loc=0, scale=(2*jak.v_err)**2,
    )
    b = np.random.normal(
        loc=0, scale=(2*jak.v_err)**2,
    )
    return a + 1j * b

In [ ]:
def distrib_normal_xr(da):
    a = np.random.normal(
        loc=0, scale=(2*da)**2,
    )
    b = np.random.normal(
        loc=0, scale=(2*da)**2,
    )
    return a + 1j * b


In [ ]:
distrib_normal_xr(jak.vx_err)

In [ ]:
jak["vx_err"].plot(vmin=0, vmax=10)

In [ ]:
        normals = {"nx": 0, "ny": 1}


In [ ]:
[key for key, _ in normals.items()]

In [ ]:
df = gp.read_file("~/Google Drive/My Drive/data/GreenlandFluxGatesAschwanden/gris-outline-vertices.gpkg")

In [ ]:
xr.Da

In [ ]:
import shapely

In [ ]:
shapely.i